### Storage System Constraints

Constraints:                                     Shadow Variables:

0 <= e[t]     (for all T in T)                 : (v_low[t])
e[t] <= maxS  (for all T in T)                 : v_up[t]
e[t] = e[t-1]+b[t] (for all t in T \ {1})      : (rho[t])
e[1] = E_init + b[1]                           : (rho[1])
e[T] = E_end                                   : (xi)

Stuff that can be left out based on Frolke chapter 5:
no networks constraints, therefore we can simply full as a single energy storage system
Discharging and charging losses aren't fullled, therefore a single variable suffices (b, which is positive when charging, negative discharging)

## FIRST DRAFT OF SPLIT-HORIZON MARKET CLEARING MODEL

### Load Packages

In [2]:
#import Pkg; Pkg.add("CSV")
#import Pkg; Pkg.add("DataFrames")
#cd("C:\\Users\\jonas\\Documents\\Jonas DTU\\MSc Thesis\\Modelling")

using JuMP
using HiGHS
using CSV
using DataFrames

### Load data from excel files and transform into matrix form


In [9]:
gen_df = CSV.read("data_gen.csv", DataFrame)
load_df = CSV.read("data_load.csv", DataFrame)

#Creating Maximum Generation matrix
P =  Array{Float64}(undef, maximum(gen_df.gen), maximum(gen_df.time))
C =  Array{Float64}(undef, maximum(gen_df.gen), maximum(gen_df.time))
for i in 1:nrow(gen_df)
    P[gen_df.gen[i],gen_df.time[i]] = gen_df.max[i]
    C[gen_df.gen[i],gen_df.time[i]] = gen_df.cost[i]
end
# Now we can acces parameters of generators as: max output = P[g,t],   Cost = C[g,t]

#Creating Maximum Consumption matrix
D = Array{Float64}(undef, maximum(load_df.load), maximum(load_df.time))
U = Array{Float64}(undef, maximum(load_df.load), maximum(load_df.time))
for i in 1:nrow(load_df)
    D[load_df.load[i],load_df.time[i]] = load_df.max[i]
    U[load_df.load[i],load_df.time[i]] = load_df.utility[i]
end
# Now we can acces parameters of loads as: max consumption = D[g,t],   Utility = U[g,t]

#Number of days
days = 2
#Hours per day
hours = 1
#Total time
time = days*hours



2

## The Full Horizon Problem

In [10]:
# Sets
T = 1:size(D)[2]
L = 1:size(D)[1]
G = 1:size(P)[1]

# Storage Parameters
S = 2.5; 
E_init = 0;

#Model 😺
full = Model(HiGHS.Optimizer)
set_silent(full)

### Variables
## Variables Market Clearing
@variable(full, b[T] )            # Energy Charged or discharged for t
@variable(full, d[L, T] >= 0)   # Demand of load l at t
@variable(full, p[G, T] >= 0)   # Production of g at t
@variable(full, e[T] >= 0)        # State of Energy at the end of T

### Constraints 
## Storage Constraints (1a-1d)
@constraint(full, Stor1[t in T],   0 <= e[t] <= S)          #1a (v_low[t] , v_up[t])
@constraint(full, Stor2[t in T;t!=1],   e[t] == e[t-1]+b[t])     #1b (rho[t])
@constraint(full, Stor3, e[1] == E_init+b[1])                 #1c (rho[1])

####### Market Clearing Formulation
@objective(full, Max, sum( sum(U[l,t]*d[l,t] for l in L) -  sum(C[g,t]*p[g,t] for g in G) for t in T) )
### Subject to
@constraint(full, Balance[t in T], sum(d[l,t] for l in L) + b[t] - sum(p[g,t] for g in G) == 0 )  # 2a
@constraint(full, Gen[g in G, t in T], 0 <= p[g,t] <= P[g,t])
@constraint(full, Load[l in L, t in T], 0 <= d[l,t] <= D[l,t])


#************************************************************************
# Solve
solution = optimize!(full)
println("Termination status: $(termination_status(full))")
#************************************************************************

if termination_status(full) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(full))")
    println("Solution:")
    for t in T
        println("The Market price in t=", t," is λ=" , dual(Balance[t]))
        println("Storage level at end of t=", t, " is e=", value(e[t]))
    end
else
    println("No optimal solution available")
end




Termination status: OPTIMAL
Optimal objective value: 27.0
Solution:
The Market price in t=1 is λ=-5.0
Storage level at end of t=1 is e=1.0
The Market price in t=2 is λ=-5.0
Storage level at end of t=2 is e=0.0


In [44]:
ρ_full = -dual(Stor2[H+1])
ρ_full

5.0

## The Future-Aware-Plus method (FAP)
##### In this problem we use the solution attained in the full-horizon problem in our objective function etc.

In [19]:
println("Cost of generator 1,2 in t=1: ", C[:,1])
println("Utility of load 1 in t=1: ", U[1])
println("Demand of load 1 in t=1: ", D[1])

Cost of generator 1,2 in t=1: [5.0, 10.0]
Utility of load 1 in t=1: 12.0
Demand of load 1 in t=1: 0.0


##### Time Horizon 1
Since there is no demand in time horizon 1, the only decision that needs to be made is whether the storage should be charged

In [49]:
#end of time horizon 1 
H = 1
# Sets
T = 1:H
L = 1:1
G = 1:2

# Storage Parameters
S = 2.5; 
E_init = 0;

#Model 😺
FAP1 = Model(HiGHS.Optimizer)
set_silent(FAP1)

### Variables
## Variables Market Clearing
@variable(FAP1, b[T] )          # Energy Charged or discharged for t
@variable(FAP1, d[L, T] >= 0)   # Demand of load l at t
@variable(FAP1, p[G, T] >= 0)   # Production of g at t
@variable(FAP1, e[T] >= 0)      # State of Energy at the end of T

### Constraints 
## Storage Constraints (1a-1d)
@constraint(FAP1, FAP_Stor1_low[t in T], 0 <= e[t] )             #1a (v_low[t])
@constraint(FAP1, FAP_Stor1_up[t in T],   e[t] <= S)             #1a (v_up[t])
@constraint(FAP1, FAP_Stor2[t in T;t!=1],   e[t] == e[t-1]+b[t])   #1b (rho[t])
@constraint(FAP1, FAP_Stor3, e[1] == E_init+b[1])                  #1c (rho[1])                  

####### Market Clearing Formulation
@objective(FAP1, Max, sum( sum(U[l,t]*d[l,t] for l in L) -  sum(C[g,t]*p[g,t] for g in G) for t in T) + e[H]*ρ_full ) # 2d 
### Subject to
@constraint(FAP1, FAP_Balance[t in T], sum(d[l,t] for l in L) + b[t] - sum(p[g,t] for g in G) == 0 )  # 2b (λ[t])
@constraint(FAP1, FAP_Gen[g in G, t in T], 0 <= p[g,t] <= P[g,t])                                     # 2c (μ_low[g,t], μ_up[g,t])
@constraint(FAP1, FAP_Load[l in L, t in T], 0 <= d[l,t] <= D[l,t])                                    # 2d (X_low[g,t], X_up[g,t])


#************************************************************************
# Solve
solution = optimize!(FAP1)
println("Termination status: $(termination_status(FAP1))")
#************************************************************************
if termination_status(FAP1) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(FAP1))")
    println("Solution:")
    for t in T
        println("The Market price in t=", t," is λ=" , dual(FAP_Balance[t]))
        println("Storage level at end of t=", t, " is e=", value(e[t]))
        println("Energy charged/discharged=", t, " is b=", value(b[t]))
        println("Consumption of con1 in t=", t, " is d=", value(d[1,t]))
        println("Production of gen1 in t=", t, " is p=", value(p[1,t]))
        println("Production of gen2 in t=", t, " is p=", value(p[2,t]))
        println("value of ρ_FAP1 in t=", t, " is p=", -dual(FAP_Stor3))
        println("value of v_low in t=", t, " is p=", -dual(FAP_Stor1_low[H]))
        println("value of v_up in t=", t, " is p=", -dual(FAP_Stor1_up[H]))
    end
else
    println("No optimal solution available")
end

### Input for time horizon 2
ρ_FAP1 = 5#-dual(FAP_Stor3)
v_low = -dual(FAP_Stor1_low[H])
v_up = -dual(FAP_Stor1_up[H])


Termination status: OPTIMAL
Optimal objective value: 0.0
Solution:
The Market price in t=1 is λ=-5.0
Storage level at end of t=1 is e=-0.0
Energy charged/discharged=1 is b=-0.0
Consumption of con1 in t=1 is d=-0.0
Production of gen1 in t=1 is p=0.0
Production of gen2 in t=1 is p=0.0
value of ρ_FAP1 in t=1 is p=5.0
value of v_low in t=1 is p=-0.0
value of v_up in t=1 is p=-0.0


-0.0

##### Time Horizon 2

In [56]:

# Sets
T2 = (H+1):time
L = 1:1
G = 1:2

# Storage Parameters
S = 2.5; 
E_init = 0;

#Model 😺
FAP2 = Model(HiGHS.Optimizer)
set_silent(FAP2)

### Variables
## Variables Market Clearing
@variable(FAP2, b[T2] )          # Energy Charged or discharged for t (Positive when charging)
@variable(FAP2, d[L, T2] >= 0)   # Demand of load l at t
@variable(FAP2, p[G, T2] >= 0)   # Production of g at t
@variable(FAP2, e[H:time] >= 0)  # State of Energy at the end of each t
@variable(FAP2, e_H >= 0)        # State of energy at end of time horizon 1

### Constraints 
## Storage Constraints (1a-1d)
@constraint(FAP2, FAP_Stor1[t in H:time],   0 <= e[t] <= S)             #1a (v_low[t] , v_up[t])
@constraint(FAP2, FAP_Stor2[t in T2;t!=T2[1]],   e[t] == e[t-1]+b[t])   #1b (rho[t])
@constraint(FAP2, FAP_Stor3, e[H+1] == e[H]+b[H+1])                  #1c (rho[1])                  

####### Market Clearing Formulation
@objective(FAP2, Max, sum( sum(U[l,t]*d[l,t] for l in L) -  sum(C[g,t]*p[g,t] for g in G) for t in T2) + e[H]*(ρ_FAP1-v_low-v_up) ) # 2d 
### Subject to
@constraint(FAP2, FAP_Balance[t in T2], sum(d[l,t] for l in L) + b[t] - sum(p[g,t] for g in G) == 0 )  # 2b (λ[t])
@constraint(FAP2, FAP_Gen[g in G, t in T2], 0 <= p[g,t] <= P[g,t])                                     # 2c (μ_low[g,t], μ_up[g,t])
@constraint(FAP2, FAP_Load[l in L, t in T2], 0 <= d[l,t] <= D[l,t])                                    # 2d (X_low[g,t], X_up[g,t])


#************************************************************************
# Solve
solution = optimize!(FAP2)
println("Termination status: $(termination_status(FAP2))")
#************************************************************************
if termination_status(FAP2) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(FAP2))")
    println("Solution:")
    println("The storage level at t=H was: ", value(e[H]))
    for t in T2
        println("The Market price in t=", t," is λ=" , dual(FAP_Balance[t]))
        println("Storage level at end of t=", t, " is e=", value(e[t]))
        println("Energy charged/discharged in t=", t, " is b=", value(b[t]))
        println("Consumption of con1 in t=", t, " is d=", value(d[1,t]))
        println("Production of gen1 in t=", t, " is p=", value(p[1,t]))
        println("Production of gen2 in t=", t, " is p=", value(p[2,t]))
    end
else
    println("No optimal solution available")
end


Termination status: OPTIMAL
Optimal objective value: 47.5
Solution:
The storage level at t=H was: 2.5
The Market price in t=2 is λ=-2.0
Storage level at end of t=2 is e=0.0
Energy charged/discharged in t=2 is b=-2.5
Consumption of con1 in t=2 is d=3.0
Production of gen1 in t=2 is p=0.5
Production of gen2 in t=2 is p=0.0
